In [16]:
%cd ~/spikes/sudoku
from copy import deepcopy
import os
from getpass import getpass

import sudoku.display
import sudoku.validation
import sudoku.construction
from sudoku.display import display_sudoku, display_sudoku_comparison
from sudoku.validation import is_proposed_solution_valid, is_single_cell_proposed, analysis_to_puzzle_solution
from sudoku.construction import construct_puzzle_solution, make_puzzle, weighted_random_choice

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

/Users/allanniemerg/spikes/sudoku


/Users/allanniemerg/.pyenv/versions/3.10.12/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/Users/allanniemerg/.pyenv/versions/3.10.12/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [17]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

 ········


### Create a Puzzle

In [7]:
#Create puzzle
solution = construct_puzzle_solution()
puzzle = make_puzzle(solution)
puzzle

[[1, 0, 0, 0], [0, 4, 0, 0], [0, 3, 1, 0], [2, 0, 4, 0]]

In [8]:
display_sudoku(puzzle)

1,,,
,4,,
,3,1,
2,,4,


In [30]:
from IPython.display import HTML, display
def display_sudoku(grid):
    # Table start with overall styling
    table_html = '<table cellspacing="0" cellpadding="5" style="border-collapse: collapse; border: 2px solid black;">'
    
    for i, row in enumerate(grid):
        table_html += '<tr style="background-color: white;">'
        for j, cell in enumerate(row):
            # Determine cell borders
            border_styles = []
            if j == 2:
                border_styles.append("border-left: 2px solid black;")
            if i == 2:
                border_styles.append("border-top: 2px solid black;")
            
            # Set cell style including size, alignment, and borders
            if not border_styles:  # if there are no thick borders, set the thin ones to gray
                cell_style = "width: 15px; height: 15px; text-align: center; border: 1px solid gray; font-size: 18px;"
            else:
                cell_style = "width: 15px; height: 15px; text-align: center; border: 1px solid gray; font-size: 18px; " + ' '.join(border_styles)
            
            table_html += '<td style="{}">{}</td>'.format(cell_style, cell if cell != 0 else '&nbsp;')
        table_html += '</tr>'
    table_html += '</table>'
    display(HTML(table_html))


In [31]:
display_sudoku(puzzle)

1,,,
,4,,
,3,1,
2,,4,


### Have GPT-3.5 solve a cell

In [11]:
brief_analysis = '''We are working on the following sudoku puzzle (each sub-list represents a row):
{}

You are a sudoku tutor. Create a brief analysis that finds an unsolved cell and solves it. 
Do not repeat the puzzle (which the student has seen). Just solve one cell that currently has a zero.
I suggest you start by examining which rows, columns, or regions have the most cells 
already solved. You can use this to identify one or more cells that are not currently solved but may be 
solvable from the available information. Then identify the solution to that cell. 

Your analysis must then solve ONLY one cell by replacing 0 with the correct number. Please don't include the 
puzzle in your analysis, we will provide that to the student seperately. 

Example puzzle: 
[[0, 0, 0, 0], [0, 0, 3, 2], [1, 0, 0, 0], [2, 0, 1, 4]]

Your analysis could look like this:
The row with the most solved cells is row 4 with numbers: 1 2 and 4. Because each row must contain the digits 1-4, 
the unsolved cell must be 3. Therefore row 4, column 2 is the number 3.'''

In [22]:
model_name = "gpt-3.5-turbo-1106"
model = ChatOpenAI(model=model_name)
prompt = brief_analysis.format(puzzle)
message = model.invoke([   
    HumanMessage(content=prompt)
])
print(message.content)

The column with the most solved cells is column 2 with numbers: 4 and 3. Because each column must contain the digits 1-4, the unsolved cell must be 1. Therefore row 1, column 2 is the number 1.


### Checking the model's output

In [23]:
extraction_model = ChatOpenAI(model="gpt-4-0613")
proposed = analysis_to_puzzle_solution(extraction_model, puzzle, message.content)
proposed

/Users/allanniemerg/.pyenv/versions/3.10.12/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[[1, 1, 0, 0], [0, 4, 0, 0], [0, 3, 1, 0], [2, 0, 4, 0]]

In [25]:
display_sudoku_comparison(proposed, puzzle) 
if is_proposed_solution_valid(puzzle, solution, proposed):
    print("Correct!")
else:
    print("Incorrect!")

1,1,,
,4,,
,3,1,
2,,4,


Incorrect!


In [32]:
is_proposed_solution_valid(puzzle, solution, proposed)

False